## 노인복지관 data

In [2]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [8]:
welfare_center = pd.read_csv(
    filepath_or_buffer = "../data/노인복지관 데이터.csv", 
    encoding = "cp949"
    )
welfare_center

,연번,시설명,주소,연락처,등록회원수
0,1,의창노인복지관,경상남도 창원시 의창구 서곡길15,055-712-0235,12017
1,2,성산노인복지관,경상남도 창원시 성산구 창이대로 781,055-712-0195,11886
2,3,마산합포노인복지관,경상남도 창원시 마산합포구 동서남1길 5,055-712-0215,15732
3,4,금강노인종합복지관(본관),경상남도 창원시 마산합포구 문화북4길 15,055-221-8445,5656
4,5,금강노인문화센터(분관),경상남도 창원시 마산회원구 회원동로 65,055-247-8445,2621
5,6,마산회원노인복지관,경상남도 창원시 마산회원구 삼호로 227,055-713-6890,7601
6,7,진해노인복지관,경상남도 창원시 진해구 천자로 434,055-544-7155,7178
7,8,진해서부노인복지관,경상남도 창원시 진해구 태평동 96,055-545-3100,4244
8,9,상락원,경상남도 진주시 판문오동길 115번길 62-33,055-749-6859,4912
9,10,청락원,경상남도 진주시 모덕로 47번길 40,055-749-6864,8714


In [9]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in welfare_center.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)

# 결과 확인
coord_df

,경도,위도
0,128.648050434,35.252740952
1,128.705180187,35.216978958
2,128.571084418,35.202361703
3,128.558797219,35.188373762
4,128.574900962,35.220150753
5,128.569577599,35.230581880
6,128.711687236,35.134609003
7,128.669316070,35.144478401
8,128.033214757,35.183482265
9,128.102742042,35.185822637


In [10]:
# 데이터 합치기
welfare_center = pd.concat([welfare_center, coord_df], axis=1)
welfare_center

,연번,시설명,주소,연락처,등록회원수,경도,위도
0,1,의창노인복지관,경상남도 창원시 의창구 서곡길15,055-712-0235,12017,128.648050434,35.252740952
1,2,성산노인복지관,경상남도 창원시 성산구 창이대로 781,055-712-0195,11886,128.705180187,35.216978958
2,3,마산합포노인복지관,경상남도 창원시 마산합포구 동서남1길 5,055-712-0215,15732,128.571084418,35.202361703
3,4,금강노인종합복지관(본관),경상남도 창원시 마산합포구 문화북4길 15,055-221-8445,5656,128.558797219,35.188373762
4,5,금강노인문화센터(분관),경상남도 창원시 마산회원구 회원동로 65,055-247-8445,2621,128.574900962,35.220150753
5,6,마산회원노인복지관,경상남도 창원시 마산회원구 삼호로 227,055-713-6890,7601,128.569577599,35.230581880
6,7,진해노인복지관,경상남도 창원시 진해구 천자로 434,055-544-7155,7178,128.711687236,35.134609003
7,8,진해서부노인복지관,경상남도 창원시 진해구 태평동 96,055-545-3100,4244,128.669316070,35.144478401
8,9,상락원,경상남도 진주시 판문오동길 115번길 62-33,055-749-6859,4912,128.033214757,35.183482265
9,10,청락원,경상남도 진주시 모덕로 47번길 40,055-749-6864,8714,128.102742042,35.185822637


In [11]:
welfare_center = welfare_center[['시설명','주소','경도','위도']]
welfare_center

,시설명,주소,경도,위도
0,의창노인복지관,경상남도 창원시 의창구 서곡길15,128.648050434,35.252740952
1,성산노인복지관,경상남도 창원시 성산구 창이대로 781,128.705180187,35.216978958
2,마산합포노인복지관,경상남도 창원시 마산합포구 동서남1길 5,128.571084418,35.202361703
3,금강노인종합복지관(본관),경상남도 창원시 마산합포구 문화북4길 15,128.558797219,35.188373762
4,금강노인문화센터(분관),경상남도 창원시 마산회원구 회원동로 65,128.574900962,35.220150753
5,마산회원노인복지관,경상남도 창원시 마산회원구 삼호로 227,128.569577599,35.230581880
6,진해노인복지관,경상남도 창원시 진해구 천자로 434,128.711687236,35.134609003
7,진해서부노인복지관,경상남도 창원시 진해구 태평동 96,128.669316070,35.144478401
8,상락원,경상남도 진주시 판문오동길 115번길 62-33,128.033214757,35.183482265
9,청락원,경상남도 진주시 모덕로 47번길 40,128.102742042,35.185822637


## 사회복지관 데이터

In [21]:
settlement = pd.read_csv(
    filepath_or_buffer = "../data/사회복지관 데이터.csv", 
    encoding = "cp949"
    )
settlement = settlement[settlement["시도"] == "경남"].reset_index(drop=True)

,연번,시도,기관명,주소,전화번호,운영주체,법인유형
0,105,경남,(경남)서부종합사회복지관,경남 사천시 곤양면 서정리 998-1,055-831-2482,사천시,지자체(직영)
1,106,경남,거제시종합사회복지관,경남 거제시 양정1길 45,055-636-0303,거제시희망복지재단,재단법인(직영)
2,107,경남,거창군종합사회복지관,경남 거창군 거창읍 아림로 66,055-940-8750,거창군,지자체(직영)
3,108,경남,경남종합사회복지관,경남 창원시 마산회원구 팔용로 272,055-298-8600,인애복지재단,사회복지법인
4,109,경남,고성군종합사회복지관,경남 고성군 고성읍 동외로 156번길 36,055-670-5913,고성군청,지자체(직영)
5,110,경남,김해시구산사회복지관,경남 김해시 가락로222(구산동),055-310-8470,김해시복지재단,재단법인
6,111,경남,김해시종합사회복지관,경남 김해시 분성로 227,055-329-6336,인제대학교 산학협력단,학교법인
7,112,경남,남해군종합사회복지관,경남 남해군 남해읍 망운로 32,055-860-3819,남해군,지자체(직영)
8,113,경남,내서종합사회복지관,경남 창원특례시 마산회원구 내서읍 상곡로 49,055-231-8017,통도사자비원,사회복지법인
9,114,경남,도남사회복지관,경남 통영시 발개로 153 도남사회복지관,055-645-0645,통영YWCA,사단법인


In [15]:
# API 엔드포인트
apiurl = "https://api.vworld.kr/req/address?"

# API 키
api_key = "8B7A9BDC-2FB8-3A2E-8F1C-359C7D1882B8"

# # 주소가 포함된 데이터프레임 예시
# data = {
#     "주소": ["서울특별시 강남구 테헤란로 123", "부산광역시 해운대구 우동 123"]
# }

# df = pd.DataFrame(data)

# 좌표 정보를 저장할 빈 리스트 생성
coordinates = []


# 데이터프레임의 각 행을 순회하며 주소를 좌표로 변환
for index, row in settlement.iterrows():
    address = row['주소']
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": api_key
    }
    response = requests.get(apiurl, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            x = data['response']['result']['point']['x']
            y = data['response']['result']['point']['y']
            coordinates.append({"경도": x, "위도": y})
        else:
            coordinates.append({"경도": None, "위도": None})
    else:
        coordinates.append({"경도": None, "위도": None})

# 결과를 데이터프레임으로 변환
coord_df = pd.DataFrame(coordinates)


In [22]:
# 데이터 합치기
settlement = pd.concat([settlement, coord_df], axis=1)
settlement.rename(columns={'기관명':'시설명'}, inplace=True)
settlement

,연번,시도,시설명,주소,전화번호,운영주체,법인유형,경도,위도
0,105,경남,(경남)서부종합사회복지관,경남 사천시 곤양면 서정리 998-1,055-831-2482,사천시,지자체(직영),None,None
1,106,경남,거제시종합사회복지관,경남 거제시 양정1길 45,055-636-0303,거제시희망복지재단,재단법인(직영),128.646102596,34.869254523
2,107,경남,거창군종합사회복지관,경남 거창군 거창읍 아림로 66,055-940-8750,거창군,지자체(직영),127.909833028,35.689069772
3,108,경남,경남종합사회복지관,경남 창원시 마산회원구 팔용로 272,055-298-8600,인애복지재단,사회복지법인,128.600952038,35.248278631
4,109,경남,고성군종합사회복지관,경남 고성군 고성읍 동외로 156번길 36,055-670-5913,고성군청,지자체(직영),128.327830470,34.975020330
5,110,경남,김해시구산사회복지관,경남 김해시 가락로222(구산동),055-310-8470,김해시복지재단,재단법인,128.877033705,35.246020252
6,111,경남,김해시종합사회복지관,경남 김해시 분성로 227,055-329-6336,인제대학교 산학협력단,학교법인,128.870732100,35.232251048
7,112,경남,남해군종합사회복지관,경남 남해군 남해읍 망운로 32,055-860-3819,남해군,지자체(직영),127.890429896,34.838533839
8,113,경남,내서종합사회복지관,경남 창원특례시 마산회원구 내서읍 상곡로 49,055-231-8017,통도사자비원,사회복지법인,None,None
9,114,경남,도남사회복지관,경남 통영시 발개로 153 도남사회복지관,055-645-0645,통영YWCA,사단법인,128.420971313,34.826797439


In [23]:
settlement = settlement[['시설명','주소','경도','위도']]
settlement

,시설명,주소,경도,위도
0,(경남)서부종합사회복지관,경남 사천시 곤양면 서정리 998-1,None,None
1,거제시종합사회복지관,경남 거제시 양정1길 45,128.646102596,34.869254523
2,거창군종합사회복지관,경남 거창군 거창읍 아림로 66,127.909833028,35.689069772
3,경남종합사회복지관,경남 창원시 마산회원구 팔용로 272,128.600952038,35.248278631
4,고성군종합사회복지관,경남 고성군 고성읍 동외로 156번길 36,128.327830470,34.975020330
5,김해시구산사회복지관,경남 김해시 가락로222(구산동),128.877033705,35.246020252
6,김해시종합사회복지관,경남 김해시 분성로 227,128.870732100,35.232251048
7,남해군종합사회복지관,경남 남해군 남해읍 망운로 32,127.890429896,34.838533839
8,내서종합사회복지관,경남 창원특례시 마산회원구 내서읍 상곡로 49,None,None
9,도남사회복지관,경남 통영시 발개로 153 도남사회복지관,128.420971313,34.826797439


In [24]:
infrastructure = pd.concat([welfare_center, settlement],axis=0).reset_index(drop=[True])
infrastructure

,시설명,주소,경도,위도
0,의창노인복지관,경상남도 창원시 의창구 서곡길15,128.648050434,35.252740952
1,성산노인복지관,경상남도 창원시 성산구 창이대로 781,128.705180187,35.216978958
2,마산합포노인복지관,경상남도 창원시 마산합포구 동서남1길 5,128.571084418,35.202361703
3,금강노인종합복지관(본관),경상남도 창원시 마산합포구 문화북4길 15,128.558797219,35.188373762
4,금강노인문화센터(분관),경상남도 창원시 마산회원구 회원동로 65,128.574900962,35.220150753
5,마산회원노인복지관,경상남도 창원시 마산회원구 삼호로 227,128.569577599,35.230581880
6,진해노인복지관,경상남도 창원시 진해구 천자로 434,128.711687236,35.134609003
7,진해서부노인복지관,경상남도 창원시 진해구 태평동 96,128.669316070,35.144478401
8,상락원,경상남도 진주시 판문오동길 115번길 62-33,128.033214757,35.183482265
9,청락원,경상남도 진주시 모덕로 47번길 40,128.102742042,35.185822637


In [25]:
infrastructure.to_csv("../data/생활인프라.csv", index=False)